<a href="https://colab.research.google.com/github/hhodek/Deep-Learning-Challenge/blob/main/AlphabetSoupOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [77]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
!pip install keras-tuner
import keras_tuner as kt

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head(3)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0


In [78]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop([ 'NAME'], axis=1)

In [79]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()
print(unique_counts)

EIN                       34299
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64


In [80]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [81]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# Choose a cutoff value (you can adjust this as needed)
cutoff_value = 500  # For example, setting a cutoff value of 500

# Get application types with frequencies below the cutoff value
low_freq_application_types = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Create a list of application types to be replaced with "Other"
application_types_to_replace = low_freq_application_types

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [82]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [83]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts_gt1 = classification_counts[classification_counts > 1]
print(classification_counts_gt1)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [84]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# Choose a cutoff value
cutoff_value = 1000

# Get classifications with frequencies below the cutoff value
low_freq_classifications = classification_counts[classification_counts < cutoff_value].index.tolist()

# Create a list of classifications to be replaced with "Other"
classifications_to_replace = low_freq_classifications

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [85]:
# Convert categorical data to numeric with `pd.get_dummies`
# Specify the categorical columns to convert
categorical_columns = ['APPLICATION_TYPE', 'AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'CLASSIFICATION']

# Create dummy variables for the specified columns
application_df = pd.get_dummies(application_df, columns=categorical_columns)



In [86]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns=['IS_SUCCESSFUL'])  # Features (excluding the target column)
y = application_df['IS_SUCCESSFUL']  # Target variable

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [87]:
import sklearn as skl

# Create scaler instance
scaler = skl.preprocessing.StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [88]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])

    # Number of input features
    input_dim = X_train_scaled.shape[1]  # Assumes the shape of X_train_scaled

    # Number of hidden layers
    num_layers = hp.Int('num_layers', 1, 6)

    # Number of neurons in each hidden layer
    for i in range(num_layers):
        nn_model.add(tf.keras.layers.Dense(
            units=hp.Int(f'units_{i}', min_value=1, max_value=10, step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [89]:
# Initialize the Keras Tuner
# Initialize the Keras Tuner
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",  # Use "val_accuracy" if you have a validation set
    max_epochs=100,  # Set the number of epochs to 100
    factor=3,  # Factor for reducing the number of models
)


Reloading Tuner from ./untitled_project/tuner0.json


In [90]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test))


In [91]:
# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]


In [92]:
# Access the values of the best hyperparameters
best_activation = best_hps.get('activation')
best_num_layers = best_hps.get('num_layers')
best_units = [best_hps.get(f'units_{i}') for i in range(best_num_layers)]

print("Best Activation Function:", best_activation)
print("Best Number of Hidden Layers:", best_num_layers)
print("Best Number of Neurons in Each Hidden Layer:", best_units)

Best Activation Function: tanh
Best Number of Hidden Layers: 2
Best Number of Neurons in Each Hidden Layer: [5, 1]


In [93]:
# Define the model using the best hyperparameters
def create_best_model():
    nn_model = tf.keras.models.Sequential()

    # Set the activation function based on the best hyperparameter
    activation = best_activation

    # Number of input features
    input_dim = X_train_scaled.shape[1]  # Assumes the shape of X_train_scaled

    # Number of hidden layers
    num_layers = best_num_layers

    # Number of neurons in each hidden layer based on the best hyperparameters
    for i in range(num_layers):
        nn_model.add(tf.keras.layers.Dense(
            units=best_units[i],
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [94]:
# Create the best model
best_model = create_best_model()


In [95]:
# Train the best model with the fixed number of epochs (e.g., 100)
best_model.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test))

Epoch 1/100
804/804 [==============================] - 4s 4ms/step - loss: 0.6276 - accuracy: 0.6660 - val_loss: 0.5814 - val_accuracy: 0.7297
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5759 - accuracy: 0.7278 - val_loss: 0.5726 - val_accuracy: 0.7304
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5704 - accuracy: 0.7294 - val_loss: 0.5688 - val_accuracy: 0.7318
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5661 - accuracy: 0.7305 - val_loss: 0.5651 - val_accuracy: 0.7291
Epoch 5/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5619 - accuracy: 0.7309 - val_loss: 0.5624 - val_accuracy: 0.7308
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5595 - accuracy: 0.7304 - val_loss: 0.5601 - val_accuracy: 0.7291
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5576 - accuracy: 0.7309 - val_loss: 0.5594 - val_accuracy: 0.7285

In [73]:
# Evaluate the model using the test data - With 100 epochs (-.001 from original)
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5560 - accuracy: 0.7285 - 410ms/epoch - 2ms/step
Loss: 0.5560060739517212, Accuracy: 0.7285131216049194


In [75]:
# Evaluate the model using the test data - With 50 epochs (-.003 from original)
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5558 - accuracy: 0.7284 - 493ms/epoch - 2ms/step
Loss: 0.555846095085144, Accuracy: 0.728396475315094


In [96]:
# Evaluate the model using the test data - With 100 epochs and no EIN
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5521 - accuracy: 0.7328 - 451ms/epoch - 2ms/step
Loss: 0.5521038174629211, Accuracy: 0.7328279614448547


In [99]:
from google.colab import files

# Define the file path for the desktop directory
desktop_path = os.path.expanduser("~/Desktop")  # "~" represents the user's home directory

# Define the complete file path including the filename
file_path = os.path.join(desktop_path, "AlphabetSoupCharity_Optimization.h5")

# Save the model to the specified file path
best_model.save(file_path)

# Use the `files` module to download the file to your local machine (desktop)
files.download(file_path)

# Print a message for confirmation
print(f"Model saved to: {file_path} and downloaded to your local machine.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Model saved to: /root/Desktop/AlphabetSoupCharity_Optimization.h5 and downloaded to your local machine.
